In [1]:
#2 Berts + GPT2 + 2RNN (In the ratio : 28 bert + 27 bert + 16 rnn + 19 gpt2 + 11 rnn)

In [2]:
from __future__ import absolute_import
from __future__ import division
from __future__ import print_function

import fastai
from fastai.train import Learner
from fastai.train import DataBunch
from fastai.callbacks import *
from fastai.basic_data import DatasetType
from keras.preprocessing import text, sequence
import torch
from torch import nn
from torch.utils import data
from torch.nn import functional as F

import os
import gc
import sys
import collections
import csv
import pandas as pd
import numpy as np
import tensorflow as tf
import random
import time
import warnings
#from tqdm import tqdm, tqdm_notebook
#import fastprogress
#from fastprogress import force_console_behavior
#from tqdm._tqdm_notebook import tqdm_notebook as tqdm

import torch.utils.data

#tqdm.pandas()


Using TensorFlow backend.


In [3]:
package_dir = '../input/jigsawdatasets/pytorch-pretrained-gpt2/pytorch-pretrained-gpt2/'
#sys.path.append(package_dir)
sys.path.insert(1,package_dir)

import torch.utils.data

from pytorch_pretrained_bert import BertTokenizer, BertForSequenceClassification, BertAdam
from pytorch_pretrained_bert import BertConfig

from pytorch_pretrained_bert import GPT2Tokenizer, GPT2ClassificationHeadModel
from pytorch_pretrained_bert import OpenAIAdam
from pytorch_pretrained_bert import GPT2Config

# # disable progress bars when submitting
# def is_interactive():
#    return 'SHLVL' not in os.environ

# if not is_interactive():
#     def nop(it, *a, **k):
#         return it
#     tqdm = nop
#     fastprogress.fastprogress.NO_BAR = True
#     master_bar, progress_bar = force_console_behavior()
#     fastai.basic_train.master_bar, fastai.basic_train.progress_bar = master_bar, progress_bar


In [4]:
# package_dir = '../input/jigsawdatasets/pytorch-pretrained-gpt2/pytorch_pretrained_gpt2/pytorch_pretrained_bert'
# sys.path.insert(1,package_dir)

# from pytorch_pretrained_bert  import GPT2Tokenizer, GPT2ClassificationHeadModel
# # from pytorch_pretrained_bert import OpenAIAdam
# # from pytorch_pretrained_bert import GPT2Config


In [5]:


warnings.filterwarnings(action='once')
device = torch.device('cuda')

    
def seed_everything(seed=123):
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.backends.cudnn.deterministic = True
seed_everything()

In [6]:
contraction_mapping = {
    "Trump's" : 'trump is',"'cause": 'because',',cause': 'because',';cause': 'because',"ain't": 'am not','ain,t': 'am not',
    'ain;t': 'am not','ain´t': 'am not','ain’t': 'am not',"aren't": 'are not',
    'aren,t': 'are not','aren;t': 'are not','aren´t': 'are not','aren’t': 'are not',"can't": 'cannot',"can't've": 'cannot have','can,t': 'cannot','can,t,ve': 'cannot have',
    'can;t': 'cannot','can;t;ve': 'cannot have',
    'can´t': 'cannot','can´t´ve': 'cannot have','can’t': 'cannot','can’t’ve': 'cannot have',
    "could've": 'could have','could,ve': 'could have','could;ve': 'could have',"couldn't": 'could not',"couldn't've": 'could not have','couldn,t': 'could not','couldn,t,ve': 'could not have','couldn;t': 'could not',
    'couldn;t;ve': 'could not have','couldn´t': 'could not',
    'couldn´t´ve': 'could not have','couldn’t': 'could not','couldn’t’ve': 'could not have','could´ve': 'could have',
    'could’ve': 'could have',"didn't": 'did not','didn,t': 'did not','didn;t': 'did not','didn´t': 'did not',
    'didn’t': 'did not',"doesn't": 'does not','doesn,t': 'does not','doesn;t': 'does not','doesn´t': 'does not',
    'doesn’t': 'does not',"don't": 'do not','don,t': 'do not','don;t': 'do not','don´t': 'do not','don’t': 'do not',
    "hadn't": 'had not',"hadn't've": 'had not have','hadn,t': 'had not','hadn,t,ve': 'had not have','hadn;t': 'had not',
    'hadn;t;ve': 'had not have','hadn´t': 'had not','hadn´t´ve': 'had not have','hadn’t': 'had not','hadn’t’ve': 'had not have',"hasn't": 'has not','hasn,t': 'has not','hasn;t': 'has not','hasn´t': 'has not','hasn’t': 'has not',
    "haven't": 'have not','haven,t': 'have not','haven;t': 'have not','haven´t': 'have not','haven’t': 'have not',"he'd": 'he would',
    "he'd've": 'he would have',"he'll": 'he will',
    "he's": 'he is','he,d': 'he would','he,d,ve': 'he would have','he,ll': 'he will','he,s': 'he is','he;d': 'he would',
    'he;d;ve': 'he would have','he;ll': 'he will','he;s': 'he is','he´d': 'he would','he´d´ve': 'he would have','he´ll': 'he will',
    'he´s': 'he is','he’d': 'he would','he’d’ve': 'he would have','he’ll': 'he will','he’s': 'he is',"how'd": 'how did',"how'll": 'how will',
    "how's": 'how is','how,d': 'how did','how,ll': 'how will','how,s': 'how is','how;d': 'how did','how;ll': 'how will',
    'how;s': 'how is','how´d': 'how did','how´ll': 'how will','how´s': 'how is','how’d': 'how did','how’ll': 'how will',
    'how’s': 'how is',"i'd": 'i would',"i'll": 'i will',"i'm": 'i am',"i've": 'i have','i,d': 'i would','i,ll': 'i will',
    'i,m': 'i am','i,ve': 'i have','i;d': 'i would','i;ll': 'i will','i;m': 'i am','i;ve': 'i have',"isn't": 'is not',
    'isn,t': 'is not','isn;t': 'is not','isn´t': 'is not','isn’t': 'is not',"it'd": 'it would',"it'll": 'it will',"It's":'it is',
    "it's": 'it is','it,d': 'it would','it,ll': 'it will','it,s': 'it is','it;d': 'it would','it;ll': 'it will','it;s': 'it is','it´d': 'it would','it´ll': 'it will','it´s': 'it is',
    'it’d': 'it would','it’ll': 'it will','it’s': 'it is',
    'i´d': 'i would','i´ll': 'i will','i´m': 'i am','i´ve': 'i have','i’d': 'i would','i’ll': 'i will','i’m': 'i am',
    'i’ve': 'i have',"let's": 'let us','let,s': 'let us','let;s': 'let us','let´s': 'let us',
    'let’s': 'let us',"ma'am": 'madam','ma,am': 'madam','ma;am': 'madam',"mayn't": 'may not','mayn,t': 'may not','mayn;t': 'may not',
    'mayn´t': 'may not','mayn’t': 'may not','ma´am': 'madam','ma’am': 'madam',"might've": 'might have','might,ve': 'might have','might;ve': 'might have',"mightn't": 'might not','mightn,t': 'might not','mightn;t': 'might not','mightn´t': 'might not',
    'mightn’t': 'might not','might´ve': 'might have','might’ve': 'might have',"must've": 'must have','must,ve': 'must have','must;ve': 'must have',
    "mustn't": 'must not','mustn,t': 'must not','mustn;t': 'must not','mustn´t': 'must not','mustn’t': 'must not','must´ve': 'must have',
    'must’ve': 'must have',"needn't": 'need not','needn,t': 'need not','needn;t': 'need not','needn´t': 'need not','needn’t': 'need not',"oughtn't": 'ought not','oughtn,t': 'ought not','oughtn;t': 'ought not',
    'oughtn´t': 'ought not','oughtn’t': 'ought not',"sha'n't": 'shall not','sha,n,t': 'shall not','sha;n;t': 'shall not',"shan't": 'shall not',
    'shan,t': 'shall not','shan;t': 'shall not','shan´t': 'shall not','shan’t': 'shall not','sha´n´t': 'shall not','sha’n’t': 'shall not',
    "she'd": 'she would',"she'll": 'she will',"she's": 'she is','she,d': 'she would','she,ll': 'she will',
    'she,s': 'she is','she;d': 'she would','she;ll': 'she will','she;s': 'she is','she´d': 'she would','she´ll': 'she will',
    'she´s': 'she is','she’d': 'she would','she’ll': 'she will','she’s': 'she is',"should've": 'should have','should,ve': 'should have','should;ve': 'should have',
    "shouldn't": 'should not','shouldn,t': 'should not','shouldn;t': 'should not','shouldn´t': 'should not','shouldn’t': 'should not','should´ve': 'should have',
    'should’ve': 'should have',"that'd": 'that would',"that's": 'that is','that,d': 'that would','that,s': 'that is','that;d': 'that would',
    'that;s': 'that is','that´d': 'that would','that´s': 'that is','that’d': 'that would','that’s': 'that is',"there'd": 'there had',
    "there's": 'there is','there,d': 'there had','there,s': 'there is','there;d': 'there had','there;s': 'there is',
    'there´d': 'there had','there´s': 'there is','there’d': 'there had','there’s': 'there is',
    "they'd": 'they would',"they'll": 'they will',"they're": 'they are',"they've": 'they have',
    'they,d': 'they would','they,ll': 'they will','they,re': 'they are','they,ve': 'they have','they;d': 'they would','they;ll': 'they will','they;re': 'they are',
    'they;ve': 'they have','they´d': 'they would','they´ll': 'they will','they´re': 'they are','they´ve': 'they have','they’d': 'they would','they’ll': 'they will',
    'they’re': 'they are','they’ve': 'they have',"wasn't": 'was not','wasn,t': 'was not','wasn;t': 'was not','wasn´t': 'was not',
    'wasn’t': 'was not',"we'd": 'we would',"we'll": 'we will',"we're": 'we are',"we've": 'we have','we,d': 'we would','we,ll': 'we will',
    'we,re': 'we are','we,ve': 'we have','we;d': 'we would','we;ll': 'we will','we;re': 'we are','we;ve': 'we have',
    "weren't": 'were not','weren,t': 'were not','weren;t': 'were not','weren´t': 'were not','weren’t': 'were not','we´d': 'we would','we´ll': 'we will',
    'we´re': 'we are','we´ve': 'we have','we’d': 'we would','we’ll': 'we will','we’re': 'we are','we’ve': 'we have',"what'll": 'what will',"what're": 'what are',"what's": 'what is',
    "what've": 'what have','what,ll': 'what will','what,re': 'what are','what,s': 'what is','what,ve': 'what have','what;ll': 'what will','what;re': 'what are',
    'what;s': 'what is','what;ve': 'what have','what´ll': 'what will',
    'what´re': 'what are','what´s': 'what is','what´ve': 'what have','what’ll': 'what will','what’re': 'what are','what’s': 'what is',
    'what’ve': 'what have',"where'd": 'where did',"where's": 'where is','where,d': 'where did','where,s': 'where is','where;d': 'where did',
    'where;s': 'where is','where´d': 'where did','where´s': 'where is','where’d': 'where did','where’s': 'where is',
    "who'll": 'who will',"who's": 'who is','who,ll': 'who will','who,s': 'who is','who;ll': 'who will','who;s': 'who is',
    'who´ll': 'who will','who´s': 'who is','who’ll': 'who will','who’s': 'who is',"won't": 'will not','won,t': 'will not','won;t': 'will not',
    'won´t': 'will not','won’t': 'will not',"wouldn't": 'would not','wouldn,t': 'would not','wouldn;t': 'would not','wouldn´t': 'would not',
    'wouldn’t': 'would not',"you'd": 'you would',"you'll": 'you will',"you're": 'you are','you,d': 'you would','you,ll': 'you will',
    'you,re': 'you are','you;d': 'you would','you;ll': 'you will',
    'you;re': 'you are','you´d': 'you would','you´ll': 'you will','you´re': 'you are','you’d': 'you would','you’ll': 'you will','you’re': 'you are',
    '´cause': 'because','’cause': 'because',"you've": "you have","could'nt": 'could not',
    "havn't": 'have not',"here’s": "here is",'i""m': 'i am',"i'am": 'i am',"i'l": "i will","i'v": 'i have',"wan't": 'want',"was'nt": "was not","who'd": "who would",
    "who're": "who are","who've": "who have","why'd": "why would","would've": "would have","y'all": "you all","y'know": "you know","you.i": "you i",
    "your'e": "you are","arn't": "are not","agains't": "against","c'mon": "common","doens't": "does not",'don""t': "do not","dosen't": "does not",
    "dosn't": "does not","shoudn't": "should not","that'll": "that will","there'll": "there will","there're": "there are",
    "this'll": "this all","u're": "you are", "ya'll": "you all","you'r": "you are","you’ve": "you have","d'int": "did not","did'nt": "did not","din't": "did not","dont't": "do not","gov't": "government",
    "i'ma": "i am","is'nt": "is not","‘I":'I',
    'ᴀɴᴅ':'and','ᴛʜᴇ':'the','ʜᴏᴍᴇ':'home','ᴜᴘ':'up','ʙʏ':'by','ᴀᴛ':'at','…and':'and','civilbeat':'civil beat',\
    'TrumpCare':'Trump care','Trumpcare':'Trump care', 'OBAMAcare':'Obama care','ᴄʜᴇᴄᴋ':'check','ғᴏʀ':'for','ᴛʜɪs':'this','ᴄᴏᴍᴘᴜᴛᴇʀ':'computer',\
    'ᴍᴏɴᴛʜ':'month','ᴡᴏʀᴋɪɴɢ':'working','ᴊᴏʙ':'job','ғʀᴏᴍ':'from','Sᴛᴀʀᴛ':'start','gubmit':'submit','CO₂':'carbon dioxide','ғɪʀsᴛ':'first',\
    'ᴇɴᴅ':'end','ᴄᴀɴ':'can','ʜᴀᴠᴇ':'have','ᴛᴏ':'to','ʟɪɴᴋ':'link','ᴏғ':'of','ʜᴏᴜʀʟʏ':'hourly','ᴡᴇᴇᴋ':'week','ᴇɴᴅ':'end','ᴇxᴛʀᴀ':'extra',\
    'Gʀᴇᴀᴛ':'great','sᴛᴜᴅᴇɴᴛs':'student','sᴛᴀʏ':'stay','ᴍᴏᴍs':'mother','ᴏʀ':'or','ᴀɴʏᴏɴᴇ':'anyone','ɴᴇᴇᴅɪɴɢ':'needing','ᴀɴ':'an','ɪɴᴄᴏᴍᴇ':'income',\
    'ʀᴇʟɪᴀʙʟᴇ':'reliable','ғɪʀsᴛ':'first','ʏᴏᴜʀ':'your','sɪɢɴɪɴɢ':'signing','ʙᴏᴛᴛᴏᴍ':'bottom','ғᴏʟʟᴏᴡɪɴɢ':'following','Mᴀᴋᴇ':'make',\
    'ᴄᴏɴɴᴇᴄᴛɪᴏɴ':'connection','ɪɴᴛᴇʀɴᴇᴛ':'internet','financialpost':'financial post', 'ʜaᴠᴇ':' have ', 'ᴄaɴ':' can ', 'Maᴋᴇ':' make ', 'ʀᴇʟɪaʙʟᴇ':' reliable ', 'ɴᴇᴇᴅ':' need ',
    'ᴏɴʟʏ':' only ', 'ᴇxᴛʀa':' extra ', 'aɴ':' an ', 'aɴʏᴏɴᴇ':' anyone ', 'sᴛaʏ':' stay ', 'Sᴛaʀᴛ':' start', 'SHOPO':'shop',
    }
mispell_dict = {'SB91':'senate bill','tRump':'trump','utmterm':'utm term','FakeNews':'fake news','Gʀᴇat':'great','ʙᴏᴛtoᴍ':'bottom','washingtontimes':'washington times','garycrum':'gary crum','htmlutmterm':'html utm term','RangerMC':'car','TFWs':'tuition fee waiver','SJWs':'social justice warrior','Koncerned':'concerned','Vinis':'vinys','Yᴏᴜ':'you','Trumpsters':'trump','Trumpian':'trump','bigly':'big league','Trumpism':'trump','Yoyou':'you','Auwe':'wonder','Drumpf':'trump','utmterm':'utm term','Brexit':'british exit','utilitas':'utilities','ᴀ':'a', '😉':'wink','😂':'joy','😀':'stuck out tongue', 'theguardian':'the guardian','deplorables':'deplorable', 'theglobeandmail':'the globe and mail', 'justiciaries': 'justiciary','creditdation': 'Accreditation','doctrne':'doctrine','fentayal': 'fentanyl','designation-': 'designation','CONartist' : 'con-artist','Mutilitated' : 'Mutilated','Obumblers': 'bumblers','negotiatiations': 'negotiations','dood-': 'dood','irakis' : 'iraki','cooerate': 'cooperate','COx':'cox','racistcomments':'racist comments','envirnmetalists': 'environmentalists',}

special_punc_mappings = {"—": "-", "–": "-", "_": "-", '”': '"', "″": '"', '“': '"', '•': '.', '−': '-',
                         "’": "'", "‘": "'", "´": "'", "`": "'", '\u200b': ' ', '\xa0': ' ','،':'','„':'',
                         '…': ' ... ', '\ufeff': ''}

spaces = ['\u200b', '\u200e', '\u202a', '\u202c', '\ufeff', '\uf0d8', '\u2061', '\x10', '\x7f', '\x9d', '\xad', '\xa0']

rare_words_mapping = {' s.p ': ' ', ' S.P ': ' ', 'U.s.p': '', 'U.S.A.': 'USA', 'u.s.a.': 'USA', 'U.S.A': 'USA','u.s.a': 'USA', 'U.S.': 'USA', 'u.s.': 'USA', ' U.S ': ' USA ', ' u.s ': ' USA ', 'U.s.': 'USA',
                      ' U.s ': 'USA', ' u.S ': ' USA ', 'fu.k': 'fuck', 'U.K.': 'UK', ' u.k ': ' UK ',' don t ': ' do not ', 'bacteries': 'batteries', ' yr old ': ' years old ', 'Ph.D': 'PhD',
                      'cau.sing': 'causing', 'Kim Jong-Un': 'The president of North Korea', 'savegely': 'savagely',
                      'Ra apist': 'Rapist', '2fifth': 'twenty fifth', '2third': 'twenty third','2nineth': 'twenty nineth', '2fourth': 'twenty fourth', '#metoo': 'MeToo',
                      'Trumpcare': 'Trump health care system', '4fifth': 'forty fifth', 'Remainers': 'remainder',
                      'Terroristan': 'terrorist', 'antibrahmin': 'anti brahmin','fuckboys': 'fuckboy', 'Fuckboys': 'fuckboy', 'Fuckboy': 'fuckboy', 'fuckgirls': 'fuck girls',
                      'fuckgirl': 'fuck girl', 'Trumpsters': 'Trump supporters', '4sixth': 'forty sixth',
                      'culturr': 'culture','weatern': 'western', '4fourth': 'forty fourth', 'emiratis': 'emirates', 'trumpers': 'Trumpster',
                      'indans': 'indians', 'mastuburate': 'masturbate', 'f**k': 'fuck', 'F**k': 'fuck', 'F**K': 'fuck',
                      ' u r ': ' you are ', ' u ': ' you ', '操你妈': 'fuck your mother', 'e.g.': 'for example',
                      'i.e.': 'in other words', '...': '.', 'et.al': 'elsewhere', 'anti-Semitic': 'anti-semitic',
                      'f***': 'fuck', 'f**': 'fuc', 'F***': 'fuck', 'F**': 'fuc','a****': 'assho', 'a**': 'ass', 'h***': 'hole', 'A****': 'assho', 'A**': 'ass', 'H***': 'hole',
                      's***': 'shit', 's**': 'shi', 'S***': 'shit', 'S**': 'shi', 'Sh**': 'shit',
                      'p****': 'pussy', 'p*ssy': 'pussy', 'P****': 'pussy','p***': 'porn', 'p*rn': 'porn', 'P***': 'porn',
                      'st*up*id': 'stupid','d***': 'dick', 'di**': 'dick', 'h*ck': 'hack',
                      'b*tch': 'bitch', 'bi*ch': 'bitch', 'bit*h': 'bitch', 'bitc*': 'bitch', 'b****': 'bitch',
                      'b***': 'bitc', 'b**': 'bit', 'b*ll': 'bull'
                      }
extra_punct = [
    ',', '.', '"', ':', ')', '(', '!', '?', '|', ';', "'", '$', '&',
    '/', '[', ']', '>', '%', '=', '#', '*', '+', '\\', '•',  '~', '@', '£',
    '·', '_', '{', '}', '©', '^', '®', '`',  '<', '→', '°', '€', '™', '›',
    '♥', '←', '×', '§', '″', '′', 'Â', '█', '½', 'à', '…', '“', '★', '”',
    '–', '●', 'â', '►', '−', '¢', '²', '¬', '░', '¶', '↑', '±', '¿', '▾',
    '═', '¦', '║', '―', '¥', '▓', '—', '‹', '─', '▒', '：', '¼', '⊕', '▼',
    '▪', '†', '■', '’', '▀', '¨', '▄', '♫', '☆', 'é', '¯', '♦', '¤', '▲',
    'è', '¸', '¾', 'Ã', '⋅', '‘', '∞', '∙', '）', '↓', '、', '│', '（', '»',
    '，', '♪', '╩', '╚', '³', '・', '╦', '╣', '╔', '╗', '▬', '❤', 'ï', 'Ø',
    '¹', '≤', '‡', '√', '«', '»', '´', 'º', '¾', '¡', '§', '£', '₤']


def correct_spelling(x, dic):
    for word in dic.keys():
        if word in x:
            x = x.replace(word, dic[word])
    return x

def correct_contraction(x, dic):
    for word in dic.keys():
        if word in x:
            x = x.replace(word, dic[word])
    return x


def pre_clean_rare_words(text):
    for rare_word in rare_words_mapping:
        if rare_word in text:
            text = text.replace(rare_word, rare_words_mapping[rare_word])

    return text

In [7]:
class LenMatchBatchSampler(torch.utils.data.BatchSampler):
    def __iter__(self):
        buckets = [[]] * 100
        yielded = 0

        for idx in self.sampler:
            count_zeros = torch.sum(self.sampler.data_source[idx][0] == 0)
            count_zeros = int(count_zeros / 64) 
            if len(buckets[count_zeros]) == 0:  buckets[count_zeros] = []

            buckets[count_zeros].append(idx)

            if len(buckets[count_zeros]) == self.batch_size:
                batch = list(buckets[count_zeros])
                #print("batch:",batch)
                yield batch
                yielded += 1
                buckets[count_zeros] = []

        batch = []
        leftover = [idx for bucket in buckets for idx in bucket]

        for idx in leftover:
            batch.append(idx)
            if len(batch) == self.batch_size:
                yielded += 1
                yield batch
                batch = []

        if len(batch) > 0 and not self.drop_last:
            yielded += 1
            yield batch

        assert len(self) == yielded, "produced an inccorect number of batches. expected %i, but yielded %i" %(len(self), yielded)

def trim_tensors(tsrs):
    max_len = torch.max(torch.sum( (tsrs[0] != 0  ), 1))
    tsrs1 = []
    if max_len > 2:         
        tsrs = [tsr[:, :max_len] if i!=1 else tsr for i,tsr in enumerate(tsrs) ]
        
    return tsrs


In [8]:
def bert_base(paths, train, test):
    
    NUM_MODELS = len(paths)
    def convert_lines(example, max_seq_length,tokenizer):
        max_seq_length -=2
        all_tokens = []
        longer = 0
        #for text in tqdm(example):
        for text in example:
            tokens_a = tokenizer.tokenize(text)
            if len(tokens_a)>max_seq_length:
                tokens_a = tokens_a[:max_seq_length]
                longer += 1
            one_token = tokenizer.convert_tokens_to_ids(["[CLS]"]+tokens_a+["[SEP]"])+[0] * (max_seq_length - len(tokens_a))
            all_tokens.append(one_token)
        return np.array(all_tokens)
    
    
    MAX_SEQUENCE_LENGTH = 220 + 2
    SEED = 1234
    BATCH_SIZE = 32
    BERT_MODEL_PATH = '../input/jigsawdatasets/uncased_l-12_h-768_a-12/uncased_L-12_H-768_A-12/'

    np.random.seed(SEED)
    torch.manual_seed(SEED)
    torch.cuda.manual_seed(SEED)
    torch.backends.cudnn.deterministic = True

    #All I need from bert_inference  is bert_config.json
    #bert_config = BertConfig('../input/jigsawdatasets/uncased_l-12_h-768_a-12/uncased_L-12_H-768_A-12/bert_config.json')
    
    bert_config = BertConfig(vocab_size_or_config_json_file=30522, hidden_size=768,
        num_hidden_layers=12, num_attention_heads=12, intermediate_size=3072,hidden_act="gelu",
                        hidden_dropout_prob=0.1,attention_probs_dropout_prob=0.1,
                 max_position_embeddings=512, type_vocab_size=2,initializer_range=0.02 ) 
#                 layer_norm_eps=1e-12)
                
    tokenizer = BertTokenizer.from_pretrained(BERT_MODEL_PATH, cache_dir=None,do_lower_case=True)

    test_df = pd.read_csv("../input/jigsaw-unintended-bias-in-toxicity-classification/test.csv")
    test_df['id1'] = np.arange(len(test_df))
    test_df['comment_text'] = test_df['comment_text'].astype(str) 
    test_df['comment_text'] = test_df['comment_text'].apply(lambda x: correct_contraction(x, contraction_mapping))
    X_test = convert_lines(test_df["comment_text"].fillna("DUMMY_VALUE"), MAX_SEQUENCE_LENGTH, tokenizer)
    
    bert_base_preds = []
    for k in range(NUM_MODELS):
        model = BertForSequenceClassification(bert_config, num_labels=8)
        #model.load_state_dict(torch.load("../input/bertmodel-jigsaw/bert_pytorch_mymodel.bin"))
        model.load_state_dict(torch.load(paths[k]))

        model.to(device)
        for param in model.parameters():
            param.requires_grad = False
        model.eval()

        test_preds = np.zeros((len(X_test)))
        
        test_dataset = torch.utils.data.TensorDataset(torch.tensor(X_test, dtype=torch.long),
                                              torch.tensor(test_df['id1'],dtype=torch.long),
                                              torch.from_numpy(np.array(X_test>0, dtype=np.uint8)),
                                              torch.zeros(X_test.shape),  torch.zeros(X_test.shape))

        ran_sampler = torch.utils.data.RandomSampler(test_dataset)
        len_sampler = LenMatchBatchSampler(ran_sampler, batch_size=32, drop_last=False)
        test_loader = torch.utils.data.DataLoader(test_dataset,batch_sampler=len_sampler)
        
#         test = torch.utils.data.TensorDataset(torch.tensor(X_test, dtype=torch.long))
#         test_loader = torch.utils.data.DataLoader(test, batch_size=32, shuffle=False)
        #tk0 = tqdm(test_loader)
        tk0 = test_loader
        for i, x_batch in enumerate(tk0):
            tsrs = trim_tensors(x_batch)
            #pred = model(x_batch.to(device), attention_mask=(x_batch > 0).to(device), labels=None)
            #test_preds[i * 32:(i + 1) * 32] = pred[:, 0].detach().cpu().squeeze().numpy()
            b_input_ids, ids, b_input_mask, b_segment_ids,b_label = tuple(t.to(device) for t in tsrs)
            pred = model(b_input_ids,attention_mask=b_input_mask, labels=None)    
            test_preds[ids.detach().cpu().numpy()] = pred[:, 0].detach().cpu().squeeze().numpy()


        test_pred_base = torch.sigmoid(torch.tensor(test_preds)).numpy().ravel()

        bert_base_preds.append(test_pred_base)
        gc.collect()
        torch.cuda.empty_cache()
    
    return bert_base_preds

In [9]:
# def bert_large(large_model_dir_base, large_model_dir_path,test):
#     # This function is not used by this file but is still used by the Colab and
#     # people who depend on it.
#     def convert_examples_to_features(examples, label_list, max_seq_length,
#                                      tokenizer):
#         """Convert a set of `InputExample`s to a list of `InputFeatures`."""
#         features = []
#         for (ex_index, example) in enumerate(examples):
#             if ex_index % 10000 == 0:
#                 tf.logging.info("Writing example %d of %d" % (ex_index, len(examples)))

#             feature = convert_single_example(ex_index, example, label_list,
#                                      max_seq_length, tokenizer)
            
#             features.append(feature)            
#         return features
    
    

#     def convert_single_example(ex_index, example, label_list, max_seq_length,
#                                tokenizer):
#         """Converts a single `InputExample` into a single `InputFeatures`."""


#         if isinstance(example, PaddingInputExample):
#             return InputFeatures(
#                 input_ids=[0] * max_seq_length,
#                 input_mask=[0] * max_seq_length,
#                 segment_ids=[0] * max_seq_length,
#                 label_id=[0]*len(LABEL_COLUMNS),
#                 is_real_example=False)

#         label_map = {}
#         for (i, label) in enumerate(label_list):
#             label_map[label] = i

#         tokens_a = tokenizer.tokenize(example.text_a)
#         tokens_b = None
#         if example.text_b:
#             tokens_b = tokenizer.tokenize(example.text_b)

#         if tokens_b:
#             # Modifies `tokens_a` and `tokens_b` in place so that the total
#             # length is less than the specified length.
#             # Account for [CLS], [SEP], [SEP] with "- 3"
#             _truncate_seq_pair(tokens_a, tokens_b, max_seq_length - 3)
#         else:
#             # Account for [CLS] and [SEP] with "- 2"
#             if len(tokens_a) > max_seq_length - 2:
#                 tokens_a = tokens_a[0:(max_seq_length - 2)]

#         # The convention in BERT is:
#         # (a) For sequence pairs:
#         #  tokens:   [CLS] is this jack ##son ##ville ? [SEP] no it is not . [SEP]
#         #  type_ids: 0     0  0    0    0     0       0 0     1  1  1  1   1 1
#         # (b) For single sequences:
#         #  tokens:   [CLS] the dog is hairy . [SEP]
#         #  type_ids: 0     0   0   0  0     0 0
#         #
#         # Where "type_ids" are used to indicate whether this is the first
#         # sequence or the second sequence. The embedding vectors for `type=0` and
#         # `type=1` were learned during pre-training and are added to the wordpiece
#         # embedding vector (and position vector). This is not *strictly* necessary
#         # since the [SEP] token unambiguously separates the sequences, but it makes
#         # it easier for the model to learn the concept of sequences.
#         #
#         # For classification tasks, the first vector (corresponding to [CLS]) is
#         # used as the "sentence vector". Note that this only makes sense because
#         # the entire model is fine-tuned.
#         tokens = []
#         segment_ids = []
#         tokens.append("[CLS]")
#         segment_ids.append(0)
#         for token in tokens_a:
#             tokens.append(token)
#             segment_ids.append(0)
#         tokens.append("[SEP]")
#         segment_ids.append(0)

#         if tokens_b:
#             for token in tokens_b:
#                 tokens.append(token)
#                 segment_ids.append(1)
#             tokens.append("[SEP]")
#             segment_ids.append(1)

#         input_ids = tokenizer.convert_tokens_to_ids(tokens)

#         # The mask has 1 for real tokens and 0 for padding tokens. Only real
#         # tokens are attended to.
#         input_mask = [1] * len(input_ids)

#         # Zero-pad up to the sequence length.
#         while len(input_ids) < max_seq_length:
#             input_ids.append(0)
#             input_mask.append(0)
#             segment_ids.append(0)

#         assert len(input_ids) == max_seq_length
#         assert len(input_mask) == max_seq_length
#         assert len(segment_ids) == max_seq_length

#         #label_id = label_map[example.label]
#         labels_ids = []
#         #print(ex_index, example.label)
#         for label in example.label:
#             labels_ids.append(float(label))

#         if ex_index < 5:
#             tf.logging.info("*** Example ***")
#             tf.logging.info("guid: %s" % (example.guid))
#             tf.logging.info("tokens: %s" % " ".join(
#                 [tokenization.printable_text(x) for x in tokens]))
#             tf.logging.info("input_ids: %s" % " ".join([str(x) for x in input_ids]))
#             tf.logging.info("input_mask: %s" % " ".join([str(x) for x in input_mask]))
#             tf.logging.info("segment_ids: %s" % " ".join([str(x) for x in segment_ids]))
#             tf.logging.info("label: %s (id = %s)" % (example.label, labels_ids))

#         feature = InputFeatures(
#                             input_ids=input_ids,
#                             input_mask=input_mask,
#                             segment_ids=segment_ids,
#                             label_id=labels_ids,
#                             is_real_example=True)
#         return feature


#     def file_based_convert_examples_to_features(
#         examples, label_list, max_seq_length, tokenizer, output_file):
#         """Convert a set of `InputExample`s to a TFRecord file."""

#         writer = tf.python_io.TFRecordWriter(output_file)
#         def create_int_feature(values):
#             f = tf.train.Feature(int64_list=tf.train.Int64List(value=list(values)))
#             return f

#         def create_float_feature(values):
#             f = tf.train.Feature(float_list = tf.train.FloatList(value=list(values)))
#             return f

#         for (ex_index, example) in enumerate(examples):
#             if ex_index % 10000 == 0:
#                 tf.logging.info("Writing example %d of %d" % (ex_index, len(examples)))

#             feature = convert_single_example(ex_index, example, label_list,
#                                          max_seq_length, tokenizer)

# #             def create_int_feature(values):
# #                 f = tf.train.Feature(int64_list=tf.train.Int64List(value=list(values)))
# #                 return f

# #             def create_float_feature(values):
# #                 f = tf.train.Feature(float_list = tf.train.FloatList(value=list(values)))
# #                 return f

#             features = collections.OrderedDict()
#             features["input_ids"] = create_int_feature(feature.input_ids)
#             features["input_mask"] = create_int_feature(feature.input_mask)
#             features["segment_ids"] = create_int_feature(feature.segment_ids)

#             features["label_ids"] = create_float_feature(feature.label_id)

#             features["is_real_example"] = create_int_feature(
#                 [int(feature.is_real_example)])

#             tf_example = tf.train.Example(features=tf.train.Features(feature=features))
#             writer.write(tf_example.SerializeToString())
            
#         writer.close()
    
#     #del test  
#     gc.collect()
    
#     export_dir_base = '../input/tf-large-bert-inference-2/1560187254-20190610t173847z-001'
#     export_dir_base = large_model_dir_base
#     data_dir = '/kaggle/working'
#     vocab_file = '../input/jigsawdatasets/uncased_l-12_h-768_a-12/uncased_L-12_H-768_A-12/vocab.txt'


#     from tensorflow.contrib import predictor

#     predict_fn_e = predictor.from_saved_model(export_dir_base + large_model_dir_path)


#     #test=pd.read_csv('../input/jigsaw-unintended-bias-in-toxicity-classification/test.csv')
#     test['comment_text'] = test['comment_text'].replace({r'\s+$': '', r'^\s+': ''}, regex=True).replace(r'\n',  ' ', regex=True).replace(r'\t',  ' ', regex=True)

#     test.to_csv( 'test.tsv', sep='\t', index=False, header=True)

#     processor = ColaProcessor()
#     #label_list = processor.get_labels()
#     label_list = [0.0,0.0,0.0,0.0,0.0,0.0,0.0]


#     predict_examples = processor.get_test_examples(data_dir)

#     tokenizer = tokenization.FullTokenizer(vocab_file=vocab_file, do_lower_case=True)


#     max_seq_length = 222
#     #num_actual_predict_examples = len(predict_examples)
#     #predict_examples = predict_examples + predict_examples[0:24]

#     i_features = convert_examples_to_features(predict_examples, label_list, max_seq_length, tokenizer)


#     k = len(i_features)
#     input_ids_list = []
#     input_mask_list = []
#     label_ids_list = []
#     segment_ids_list = []
#     for i in range(k):
#        input_ids_list.append(i_features[i].input_ids)
#        input_mask_list.append(i_features[i].input_mask)
#        #label_ids_list.append(i_features[i].label_id)
#        segment_ids_list.append(i_features[i].segment_ids) 


#     torch.cuda.empty_cache()
#     gc.collect()

#     #prediction with batches 

# #    start = time.time()
# #     print("--------------------------------------------------------")
# #     print("Starting infer ...")
# #     print("--------------------------------------------------------")
#     batch_size = 512 #1280 #1024



#     k = (int)(np.ceil(len(predict_examples)/batch_size))
#     #tq = tqdm_notebook(range(k))
#     tq =  range(k) 
#     preds = np.array(np.zeros((len(predict_examples),8)))
#     for i in tq:
#     #    tmp = predict_fn_e({'input_ids': input_ids_list[i * batch_size:(i+1) * batch_size] , 
#     #                        'input_mask': input_mask_list[i * batch_size:(i+1) * batch_size], 
#     #                        #'label_ids': label_ids_list[i * batch_size:(i+1) * batch_size],
#     #                        'segment_ids':segment_ids_list[i * batch_size:(i+1) * batch_size] } )['probabilities']
#     #    print(tmp.shape)

#         preds[i * batch_size:(i+1) * batch_size,:] = predict_fn_e({'input_ids': input_ids_list[i * batch_size:(i+1) * batch_size] , 
#                            'input_mask': input_mask_list[i * batch_size:(i+1) * batch_size], 
#                            #'label_ids': label_ids_list[i * batch_size:(i+1) * batch_size],
#                            'segment_ids':segment_ids_list[i * batch_size:(i+1) * batch_size] } )['probabilities']
#         gc.collect()
#         torch.cuda.empty_cache()

#     end = time.time()
# #     print("--------------------------------------------------------")
# #     print("Infer complete in ", end - start, " seconds")
# #     print("--------------------------------------------------------")
    
#     del predict_fn_e,predict_examples, processor, input_ids_list, input_mask_list, segment_ids_list
#     gc.collect()
#     torch.cuda.empty_cache()
#     tf.reset_default_graph()
    
#     return preds

In [10]:
def rnn_lstm(paths, train, test):
    
    NUM_MODELS = len(paths)
    CRAWL_EMBEDDING_PATH = '../input/jigsawdatasets/crawl-300d-2M.pkl'
    GLOVE_EMBEDDING_PATH = '../input/jigsawdatasets/glove.840B.300d.pkl'

    def get_coefs(word, *arr):
        return word, np.asarray(arr, dtype='float32')

    def load_embeddings(path):
        with open(path,'rb') as f:
            emb_arr = pickle.load(f)
        return emb_arr

    def build_matrix(word_index, path):
        embedding_index = load_embeddings(path)
        embedding_matrix = np.zeros((max_features + 1, 300))
        unknown_words = []

        for word, i in word_index.items():
            if i <= max_features:
                try:
                    embedding_matrix[i] = embedding_index[word]
                except KeyError:
                    try:
                        embedding_matrix[i] = embedding_index[word.lower()]
                    except KeyError:
                        try:
                            embedding_matrix[i] = embedding_index[word.title()]
                        except KeyError:
                            unknown_words.append(word)
        return embedding_matrix, unknown_words
    
    test['comment_text'] = test['comment_text'].astype(str).fillna("DUMMY_VALUE")
    
    symbols_to_isolate = '.,?!-;*"…:—()%#$&_/@＼・ω+=”“[]^–>\\°<~•≠™ˈʊɒ∞§{}·τα❤☺ɡ|¢→̶`❥━┣┫┗Ｏ►★©―ɪ✔®\x96\x92●£♥➤´¹☕≈÷♡◐║▬′ɔː€۩۞†μ✒➥═☆ˌ◄½ʻπδηλσερνʃ✬ＳＵＰＥＲＩＴ☻±♍µº¾✓◾؟．⬅℅»Вав❣⋅¿¬♫ＣＭβ█▓▒░⇒⭐›¡₂₃❧▰▔◞▀▂▃▄▅▆▇↙γ̄″☹➡«φ⅓„✋：¥̲̅́∙‛◇✏▷❓❗¶˚˙）сиʿ✨。ɑ\x80◕！％¯−ﬂﬁ₁²ʌ¼⁴⁄₄⌠♭✘╪▶☭✭♪☔☠♂☃☎✈✌✰❆☙○‣⚓年∎ℒ▪▙☏⅛ｃａｓǀ℮¸ｗ‚∼‖ℳ❄←☼⋆ʒ⊂、⅔¨͡๏⚾⚽Φ×θ￦？（℃⏩☮⚠月✊❌⭕▸■⇌☐☑⚡☄ǫ╭∩╮，例＞ʕɐ̣Δ₀✞┈╱╲▏▕┃╰▊▋╯┳┊≥☒↑☝ɹ✅☛♩☞ＡＪＢ◔◡↓♀⬆̱ℏ\x91⠀ˤ╚↺⇤∏✾◦♬³の｜／∵∴√Ω¤☜▲↳▫‿⬇✧ｏｖｍ－２０８＇‰≤∕ˆ⚜☁'
    symbols_to_delete = '\n🍕\r🐵😑\xa0\ue014\t\uf818\uf04a\xad😢🐶️\uf0e0😜😎👊\u200b\u200e😁عدويهصقأناخلىبمغر😍💖💵Е👎😀😂\u202a\u202c🔥😄🏻💥ᴍʏʀᴇɴᴅᴏᴀᴋʜᴜʟᴛᴄᴘʙғᴊᴡɢ😋👏שלוםבי😱‼\x81エンジ故障\u2009🚌ᴵ͞🌟😊😳😧🙀😐😕\u200f👍😮😃😘אעכח💩💯⛽🚄🏼ஜ😖ᴠ🚲‐😟😈💪🙏🎯🌹😇💔😡\x7f👌ἐὶήιὲκἀίῃἴξ🙄Ｈ😠\ufeff\u2028😉😤⛺🙂\u3000تحكسة👮💙فزط😏🍾🎉😞\u2008🏾😅😭👻😥😔😓🏽🎆🍻🍽🎶🌺🤔😪\x08‑🐰🐇🐱🙆😨🙃💕𝘊𝘦𝘳𝘢𝘵𝘰𝘤𝘺𝘴𝘪𝘧𝘮𝘣💗💚地獄谷улкнПоАН🐾🐕😆ה🔗🚽歌舞伎🙈😴🏿🤗🇺🇸мυтѕ⤵🏆🎃😩\u200a🌠🐟💫💰💎эпрд\x95🖐🙅⛲🍰🤐👆🙌\u2002💛🙁👀🙊🙉\u2004ˢᵒʳʸᴼᴷᴺʷᵗʰᵉᵘ\x13🚬🤓\ue602😵άοόςέὸתמדףנרךצט😒͝🆕👅👥👄🔄🔤👉👤👶👲🔛🎓\uf0b7\uf04c\x9f\x10成都😣⏺😌🤑🌏😯ех😲Ἰᾶὁ💞🚓🔔📚🏀👐\u202d💤🍇\ue613小土豆🏡❔⁉\u202f👠》कर्मा🇹🇼🌸蔡英文🌞🎲レクサス😛外国人关系Сб💋💀🎄💜🤢َِьыгя不是\x9c\x9d🗑\u2005💃📣👿༼つ༽😰ḷЗз▱ц￼🤣卖温哥华议会下降你失去所有的钱加拿大坏税骗子🐝ツ🎅\x85🍺آإشء🎵🌎͟ἔ油别克🤡🤥😬🤧й\u2003🚀🤴ʲшчИОРФДЯМюж😝🖑ὐύύ特殊作戦群щ💨圆明园קℐ🏈😺🌍⏏ệ🍔🐮🍁🍆🍑🌮🌯🤦\u200d𝓒𝓲𝓿𝓵안영하세요ЖљКћ🍀😫🤤ῦ我出生在了可以说普通话汉语好极🎼🕺🍸🥂🗽🎇🎊🆘🤠👩🖒🚪天一家⚲\u2006⚭⚆⬭⬯⏖新✀╌🇫🇷🇩🇪🇮🇬🇧😷🇨🇦ХШ🌐\x1f杀鸡给猴看ʁ𝗪𝗵𝗲𝗻𝘆𝗼𝘂𝗿𝗮𝗹𝗶𝘇𝗯𝘁𝗰𝘀𝘅𝗽𝘄𝗱📺ϖ\u2000үսᴦᎥһͺ\u2007հ\u2001ɩｙｅ൦ｌƽｈ𝐓𝐡𝐞𝐫𝐮𝐝𝐚𝐃𝐜𝐩𝐭𝐢𝐨𝐧Ƅᴨןᑯ໐ΤᏧ௦Іᴑ܁𝐬𝐰𝐲𝐛𝐦𝐯𝐑𝐙𝐣𝐇𝐂𝐘𝟎ԜТᗞ౦〔Ꭻ𝐳𝐔𝐱𝟔𝟓𝐅🐋ﬃ💘💓ё𝘥𝘯𝘶💐🌋🌄🌅𝙬𝙖𝙨𝙤𝙣𝙡𝙮𝙘𝙠𝙚𝙙𝙜𝙧𝙥𝙩𝙪𝙗𝙞𝙝𝙛👺🐷ℋ𝐀𝐥𝐪🚶𝙢Ἱ🤘ͦ💸ج패티Ｗ𝙇ᵻ👂👃ɜ🎫\uf0a7БУі🚢🚂ગુજરાતીῆ🏃𝓬𝓻𝓴𝓮𝓽𝓼☘﴾̯﴿₽\ue807𝑻𝒆𝒍𝒕𝒉𝒓𝒖𝒂𝒏𝒅𝒔𝒎𝒗𝒊👽😙\u200cЛ‒🎾👹⎌🏒⛸公寓养宠物吗🏄🐀🚑🤷操美𝒑𝒚𝒐𝑴🤙🐒欢迎来到阿拉斯ספ𝙫🐈𝒌𝙊𝙭𝙆𝙋𝙍𝘼𝙅ﷻ🦄巨收赢得白鬼愤怒要买额ẽ🚗🐳𝟏𝐟𝟖𝟑𝟕𝒄𝟗𝐠𝙄𝙃👇锟斤拷𝗢𝟳𝟱𝟬⦁マルハニチロ株式社⛷한국어ㄸㅓ니͜ʖ𝘿𝙔₵𝒩ℯ𝒾𝓁𝒶𝓉𝓇𝓊𝓃𝓈𝓅ℴ𝒻𝒽𝓀𝓌𝒸𝓎𝙏ζ𝙟𝘃𝗺𝟮𝟭𝟯𝟲👋🦊多伦🐽🎻🎹⛓🏹🍷🦆为和中友谊祝贺与其想象对法如直接问用自己猜本传教士没积唯认识基督徒曾经让相信耶稣复活死怪他但当们聊些政治题时候战胜因圣把全堂结婚孩恐惧且栗谓这样还♾🎸🤕🤒⛑🎁批判检讨🏝🦁🙋😶쥐스탱트뤼도석유가격인상이경제황을렵게만들지않록잘관리해야합다캐나에서대마초와화약금의품런성분갈때는반드시허된사용🔫👁凸ὰ💲🗯𝙈Ἄ𝒇𝒈𝒘𝒃𝑬𝑶𝕾𝖙𝖗𝖆𝖎𝖌𝖍𝖕𝖊𝖔𝖑𝖉𝖓𝖐𝖜𝖞𝖚𝖇𝕿𝖘𝖄𝖛𝖒𝖋𝖂𝕴𝖟𝖈𝕸👑🚿💡知彼百\uf005𝙀𝒛𝑲𝑳𝑾𝒋𝟒😦𝙒𝘾𝘽🏐𝘩𝘨ὼṑ𝑱𝑹𝑫𝑵𝑪🇰🇵👾ᓇᒧᔭᐃᐧᐦᑳᐨᓃᓂᑲᐸᑭᑎᓀᐣ🐄🎈🔨🐎🤞🐸💟🎰🌝🛳点击查版🍭𝑥𝑦𝑧ＮＧ👣\uf020っ🏉ф💭🎥Ξ🐴👨🤳🦍\x0b🍩𝑯𝒒😗𝟐🏂👳🍗🕉🐲چی𝑮𝗕𝗴🍒ꜥⲣⲏ🐑⏰鉄リ事件ї💊「」\uf203\uf09a\uf222\ue608\uf202\uf099\uf469\ue607\uf410\ue600燻製シ虚偽屁理屈Г𝑩𝑰𝒀𝑺🌤𝗳𝗜𝗙𝗦𝗧🍊ὺἈἡχῖΛ⤏🇳𝒙ψՁմեռայինրւդձ冬至ὀ𝒁🔹🤚🍎𝑷🐂💅𝘬𝘱𝘸𝘷𝘐𝘭𝘓𝘖𝘹𝘲𝘫کΒώ💢ΜΟΝΑΕ🇱♲𝝈↴💒⊘Ȼ🚴🖕🖤🥘📍👈➕🚫🎨🌑🐻𝐎𝐍𝐊𝑭🤖🎎😼🕷ｇｒｎｔｉｄｕｆｂｋ𝟰🇴🇭🇻🇲𝗞𝗭𝗘𝗤👼📉🍟🍦🌈🔭《🐊🐍\uf10aლڡ🐦\U0001f92f\U0001f92a🐡💳ἱ🙇𝗸𝗟𝗠𝗷🥜さようなら🔼'

    from nltk.tokenize.treebank import TreebankWordTokenizer
    tokenizer = TreebankWordTokenizer()

    isolate_dict = {ord(c):f' {c} ' for c in symbols_to_isolate}
    remove_dict = {ord(c):f'' for c in symbols_to_delete}

    def handle_punctuation(x):
        x = x.translate(remove_dict)
        x = x.translate(isolate_dict)
        return x

    def handle_contractions(x):
        x = tokenizer.tokenize(x)
        return x
    
    def fix_quote(x):
        x = [x_[1:] if x_.startswith("'") else x_ for x_ in x]
        x = ' '.join(x)
        return x

    def preprocess(x):
        x = handle_punctuation(x)
        x = handle_contractions(x)
        x = fix_quote(x)

        x = x.replace("n't",'not')
        x = x.replace("N'T",'NOT')
        x = x.replace('tRump','Trump')
        x = x.replace("gov't","government")
        x = x.replace("Gov't","Government")
        x = x.replace("Twitler","Twitter")
        return x
    
    #x_train = train['comment_text'].progress_apply(lambda x:preprocess(x))
    x_train = train['comment_text'].apply(lambda x:preprocess(x))
    y_aux_train = train[['target', 'severe_toxicity', 'obscene', 'identity_attack', 'insult', 'threat']]
    x_test = test['comment_text'].apply(lambda x:preprocess(x))
    #x_test = test['comment_text'].progress_apply(lambda x:preprocess(x))

    TOXICITY_COLUMN = 'target'
    identity_columns = ['asian', 'atheist',
           'bisexual', 'black', 'buddhist', 'christian', 'female',
           'heterosexual', 'hindu', 'homosexual_gay_or_lesbian',
           'intellectual_or_learning_disability', 'jewish', 'latino', 'male',
           'muslim', 'other_disability', 'other_gender',
           'other_race_or_ethnicity', 'other_religion',
           'other_sexual_orientation', 'physical_disability',
           'psychiatric_or_mental_illness', 'transgender', 'white']

    subgroup_bool_train = train[identity_columns].fillna(0)>=0.5
    toxic_bool_train = train[TOXICITY_COLUMN].fillna(0)>=0.5
    subgroup_negative_mask = subgroup_bool_train.values.sum(axis=1).astype(bool) & ~toxic_bool_train
    # Overall
    weights = np.ones((len(train),))
    # Subgroup negative
    weights += subgroup_negative_mask
    loss_weight = 1.0 / weights.mean()

    #y_train = np.vstack([(train['target'].values>=0.5).astype(np.int),weights]).T
    
    max_features = 400000
    tokenizer = text.Tokenizer(num_words = max_features, filters='',lower=False, oov_token = 'UNK')
    tokenizer.fit_on_texts(list(x_train)+list(x_test))
    print(len(tokenizer.word_counts)) 

    crawl_matrix, unknown_words_crawl = build_matrix(tokenizer.word_index, CRAWL_EMBEDDING_PATH)
    print('n unknown words (crawl): ', len(unknown_words_crawl))

    glove_matrix, unknown_words_glove = build_matrix(tokenizer.word_index, GLOVE_EMBEDDING_PATH)
    print('n unknown words (glove): ', len(unknown_words_glove))

    max_features = max_features or len(tokenizer.word_index) + 1
    embedding_matrix = np.concatenate([crawl_matrix, glove_matrix], axis=-1)
    
    del crawl_matrix
    del glove_matrix
    gc.collect()

    #x_train_torch = torch.tensor(x_train, dtype=torch.long)
    #y_train_torch = torch.tensor(np.hstack([y_train, y_aux_train]), dtype=torch.float32)

    gc.collect()
    torch.cuda.empty_cache()
    
    #x_train = tokenizer.texts_to_sequences(x_train)
    x_test = tokenizer.texts_to_sequences(x_test)

    #lengths = torch.from_numpy(np.array([len(x) for x in x_train]))

    maxlen = 300
    #x_train_padded = torch.from_numpy(sequence.pad_sequences(x_train, maxlen=maxlen))
    #print(x_train_padded.shape)

    # lengths = torch.from_numpy(np.array([len(x) for x in x_train]))
    test_lengths = torch.from_numpy(np.array([len(x) for x in x_test]))
    x_test_padded = torch.from_numpy(sequence.pad_sequences(x_test, maxlen=maxlen))
    
    del x_train, x_test, tokenizer
    gc.collect()
    
    class SequenceBucketCollator():
        def __init__(self, choose_length, sequence_index, length_index, label_index=None):
            self.choose_length = choose_length
            self.sequence_index = sequence_index
            self.length_index = length_index
            self.label_index = label_index

        def __call__(self, batch):
            batch = [torch.stack(x) for x in list(zip(*batch))]

            sequences = batch[self.sequence_index]
            lengths = batch[self.length_index]

            length = self.choose_length(lengths)
            mask = torch.arange(start=maxlen, end=0, step=-1) < length
            padded_sequences = sequences[:, mask]

            batch[self.sequence_index] = padded_sequences

            if self.label_index is not None:
                return [x for i, x in enumerate(batch) if i != self.label_index], batch[self.label_index]

            return batch
        
    LSTM_UNITS = 128
    DENSE_HIDDEN_UNITS = 4 * LSTM_UNITS

    def sigmoid(x):
        return 1 / (1 + np.exp(-x))

    class SpatialDropout(nn.Dropout2d):
        def forward(self, x):
            x = x.unsqueeze(2)    # (N, T, 1, K)
            x = x.permute(0, 3, 2, 1)  # (N, K, 1, T)
            x = super(SpatialDropout, self).forward(x)  # (N, K, 1, T), some features are masked
            x = x.permute(0, 3, 2, 1)  # (N, T, 1, K)
            x = x.squeeze(2)  # (N, T, K)
            return x
        
    class NeuralNet(nn.Module):
        def __init__(self, embedding_matrix, num_aux_targets):
            super(NeuralNet, self).__init__()
            embed_size = embedding_matrix.shape[1]

            self.embedding = nn.Embedding(max_features, embed_size)
            self.embedding.weight = nn.Parameter(torch.tensor(embedding_matrix, dtype=torch.float32))
            self.embedding.weight.requires_grad = False
            self.embedding_dropout = SpatialDropout(0.2)

            self.lstm1 = nn.LSTM(embed_size, LSTM_UNITS, bidirectional=True, batch_first=True)
            self.lstm2 = nn.LSTM(LSTM_UNITS * 2, LSTM_UNITS, bidirectional=True, batch_first=True)

            self.linear1 = nn.Linear(DENSE_HIDDEN_UNITS, DENSE_HIDDEN_UNITS)
            self.linear2 = nn.Linear(DENSE_HIDDEN_UNITS, DENSE_HIDDEN_UNITS)

            self.linear_out = nn.Linear(DENSE_HIDDEN_UNITS, 1)
            self.linear_aux_out = nn.Linear(DENSE_HIDDEN_UNITS, num_aux_targets)

        def forward(self, x, lengths=None):
            h_embedding = self.embedding(x.long())
            h_embedding = self.embedding_dropout(h_embedding)

            h_lstm1, _ = self.lstm1(h_embedding)
            h_lstm2, _ = self.lstm2(h_lstm1)

            # global average pooling
            avg_pool = torch.mean(h_lstm2, 1)
            # global max pooling
            max_pool, _ = torch.max(h_lstm2, 1)

            h_conc = torch.cat((max_pool, avg_pool), 1)
            h_conc_linear1  = F.relu(self.linear1(h_conc))
            h_conc_linear2  = F.relu(self.linear2(h_conc))

            hidden = h_conc + h_conc_linear1 + h_conc_linear2

            result = self.linear_out(hidden)
            aux_result = self.linear_aux_out(hidden)
            out = torch.cat([result, aux_result], 1)

            return out

    def custom_loss(data, targets):
        ''' Define custom loss function for weighted BCE on 'target' column '''
        bce_loss_1 = nn.BCEWithLogitsLoss(weight=targets[:,1:2])(data[:,:1],targets[:,:1])
        bce_loss_2 = nn.BCEWithLogitsLoss()(data[:,1:],targets[:,2:])
        return (bce_loss_1 * loss_weight) + bce_loss_2

    #Inference No training
    #print(x_train_padded.shape)
    print(x_test_padded.shape)
    #print(y_train_torch.shape)
    
    rnn_results = []
    for i in range(NUM_MODELS):
        MODEL_DIR = paths[i]
        
        model_dict = {'fold0_models': ['0_model1_0_epoch.bin','0_model1_1_epoch.bin','0_model1_2_epoch.bin','0_model1_3_epoch.bin'], 
              'fold1_models': ['1_model1_0_epoch.bin','1_model1_1_epoch.bin','1_model1_2_epoch.bin','1_model1_3_epoch.bin'],
              'fold2_models': ['2_model1_0_epoch.bin','2_model1_1_epoch.bin','2_model1_2_epoch.bin','2_model1_3_epoch.bin'],
              'fold3_models': ['3_model1_0_epoch.bin','3_model1_1_epoch.bin','3_model1_2_epoch.bin','3_model1_3_epoch.bin'],
              'fold4_models': ['4_model1_0_epoch.bin','4_model1_1_epoch.bin','4_model1_2_epoch.bin','4_model1_3_epoch.bin'] }

        SPLITS = 5
        EPOCHS = 4
        OUTPUT_DIM = 7
        batch_size = 512

        test_dataset = data.TensorDataset(x_test_padded, test_lengths)
        test_collator = SequenceBucketCollator(lambda lenghts: lenghts.max(), sequence_index=0, length_index=1)
        test_loader = data.DataLoader(test_dataset, batch_size=batch_size, shuffle=False, collate_fn=test_collator)
        checkpoint_weights = [2 ** epoch for epoch in range(EPOCHS)]

        test_preds_all_folds = []
        
        #average predictions over 4 epochs per fold
        for fld in model_dict.keys():    
            test_preds = np.zeros((len(test_dataset),OUTPUT_DIM))
            test_preds_all_epochs = []
            for e in range(EPOCHS):
                #print(model_dict[fld][e])
                dict_load = torch.load(MODEL_DIR + model_dict[fld][e] )
                dict_load['embedding.weight'] = torch.from_numpy(embedding_matrix)
                m2 = NeuralNet(embedding_matrix,OUTPUT_DIM-1).cuda()  
                m2.load_state_dict(dict_load)
                m2.to('cuda')
                for param in m2.parameters():
                    param.requires_grad = False
                m2.eval()

                for i, x_batch in enumerate(test_loader):
                    pred = m2(x_batch[0].to('cuda'))
                    #print(pred.shape)
                    test_preds[i * batch_size:(i + 1) * batch_size] = sigmoid(pred.detach().cpu().squeeze().numpy()) #.argmax(axis=-1)

                #print(test_preds.shape)
                test_preds_all_epochs.append(test_preds)

            test_preds_all_folds.append(np.average(test_preds_all_epochs, weights=checkpoint_weights, axis=0))

        test_pred_lstm = np.mean(test_preds_all_folds, axis=0)[:, 0]
        rnn_results.append(test_pred_lstm) 
    
    #delete everything except rnn_results
    del train,test, symbols_to_isolate, symbols_to_delete,  y_aux_train, 
    del subgroup_bool_train, subgroup_negative_mask, toxic_bool_train, weights, loss_weight, 
    del embedding_matrix,  x_test_padded
    #y_train_torch, x_train_padded, y_train,
    del m2, test_dataset, test_collator, test_loader, test_preds_all_folds, test_pred_lstm, test_preds_all_epochs, 
    
    gc.collect()
    torch.cuda.empty_cache()
    return rnn_results #np.mean(rnn_results,axis=0)[:, 0]

In [11]:
TRAIN_CSV = '../input/jigsaw-unintended-bias-in-toxicity-classification/train.csv'
TEST_CSV = '../input/jigsaw-unintended-bias-in-toxicity-classification/test.csv'

train = pd.read_csv(TRAIN_CSV)  
test = pd.read_csv(TEST_CSV) 



In [12]:
def gpt2_model(paths, train, test):
    
    NUM_MODELS = len(paths)

    GPT2_MODEL_PATH = '../input/jigsawdatasets/gpt2-models_kaggle/gpt2-models_kaggle/'
    TOKENIZER_PATH = '../input/jigsawdatasets/gpt2-models_kaggle/gpt2-models_kaggle/'
    GPT2_CONFIG_PATH = '../input/jigsawdatasets/gpt2-models_kaggle/gpt2-models_kaggle/config.json'

    MAX_SEQUENCE_LENGTH = 220
    SEED = 1234
    BATCH_SIZE = 32
    np.random.seed(SEED)
    torch.manual_seed(SEED)
    torch.cuda.manual_seed(SEED)
    torch.backends.cudnn.deterministic = True
    
    def convert_lines(example, max_seq_length,tokenizer):
        max_seq_length -=2
        all_tokens = []
        longer = 0
        #for text in tqdm_notebook(example):
        for text in example:
            tokens_a = tokenizer.tokenize(text)
            if len(tokens_a)>max_seq_length:
                tokens_a = tokens_a[:max_seq_length]
                longer += 1
            one_token = tokenizer.convert_tokens_to_ids(tokens_a)+[0] * (max_seq_length - len(tokens_a))
            all_tokens.append(one_token)
        print(longer)
        return np.array(all_tokens)
    
    gpt2_config = GPT2Config(GPT2_CONFIG_PATH)
    tokenizer = GPT2Tokenizer.from_pretrained(TOKENIZER_PATH)
    
    test['comment_text'] = test['comment_text'].astype(str) 
    test['id1'] = np.arange(len(test))
    
    X_test = convert_lines(test["comment_text"].fillna("DUMMY_VALUE"), MAX_SEQUENCE_LENGTH, tokenizer)
    y_columns = ['target', 'weights', 'severe_toxicity', 'obscene', 'identity_attack', 'insult', 'threat', 'sexual_explicit']
    
    gpt2_preds = []
    for k in range(NUM_MODELS):
        model = GPT2ClassificationHeadModel.from_pretrained(GPT2_MODEL_PATH, clf_dropout=0.2, n_class=len(y_columns))
        model.load_state_dict(torch.load(paths[k]))
        model.to(device)
        for param in model.parameters():
            param.requires_grad = False
        model.eval()
    
        test_preds = np.zeros((len(X_test)))
        
        
        test_dataset = torch.utils.data.TensorDataset(torch.tensor(X_test, dtype=torch.long),
                                                  torch.tensor(test['id1'],dtype=torch.long))

        ran_sampler = torch.utils.data.RandomSampler(test_dataset)
        len_sampler = LenMatchBatchSampler(ran_sampler, batch_size=32, drop_last=False)
        test_loader = torch.utils.data.DataLoader(test_dataset,batch_sampler=len_sampler)

        #test = torch.utils.data.TensorDataset(torch.tensor(X_test, dtype=torch.long))
        #test_loader = torch.utils.data.DataLoader(test, batch_size=32, shuffle=False)
        #tk0 = tqdm(test_loader)
        tk0 = test_loader
        for i, x_batch in enumerate(tk0):
            tsrs = trim_tensors(x_batch)
            b_input_ids, ids  = tuple(t.to(device) for t in tsrs)
            pred = model(b_input_ids.to(device))
            #test_preds[i * 32:(i + 1) * 32] = pred[:, 0].detach().cpu().squeeze().numpy()
            test_preds[ids.detach().cpu().numpy()] = pred[:, 0].detach().cpu().squeeze().numpy()
    
        test_pred = torch.sigmoid(torch.tensor(test_preds)).numpy().ravel()
        
        gpt2_preds.append(test_pred)
        gc.collect()
        torch.cuda.empty_cache()
    
    
    del tokenizer, test, X_test
    gc.collect()
    torch.cuda.empty_cache()
    return gpt2_preds

In [13]:
# def gpt2_model(paths, train, test):
#     NUM_MODELS = len(paths)

#     GPT2_MODEL_PATH = '../input/jigsawdatasets/gpt2-models_kaggle/gpt2-models_kaggle/'
#     TOKENIZER_PATH = '../input/jigsawdatasets/gpt2-models_kaggle/gpt2-models_kaggle/'
#     GPT2_CONFIG_PATH = '../input/jigsawdatasets/gpt2-models_kaggle/gpt2-models_kaggle/config.json'

#     MAX_SEQUENCE_LENGTH = 220
#     SEED = 1234
#     BATCH_SIZE = 32
#     np.random.seed(SEED)
#     torch.manual_seed(SEED)
#     torch.cuda.manual_seed(SEED)
#     torch.backends.cudnn.deterministic = True
    
#     def convert_lines(example, max_seq_length,tokenizer):
#         max_seq_length -=2
#         all_tokens = []
#         longer = 0
#         for text in tqdm_notebook(example):
#             tokens_a = tokenizer.tokenize(text)
#             if len(tokens_a)>max_seq_length:
#                 tokens_a = tokens_a[:max_seq_length]
#                 longer += 1
#             one_token = tokenizer.convert_tokens_to_ids(tokens_a)+[0] * (max_seq_length - len(tokens_a))
#             all_tokens.append(one_token)
#         print(longer)
#         return np.array(all_tokens)
    
#     gpt2_config = GPT2Config(GPT2_CONFIG_PATH)
#     tokenizer = GPT2Tokenizer.from_pretrained(TOKENIZER_PATH)
    
#     test['comment_text'] = test['comment_text'].astype(str) 
#     X_test = convert_lines(test["comment_text"].fillna("DUMMY_VALUE"), MAX_SEQUENCE_LENGTH, tokenizer)
#     y_columns = ['target', 'weights', 'severe_toxicity', 'obscene', 'identity_attack', 'insult', 'threat', 'sexual_explicit']

#     gpt2_preds = []
#     for i in range(NUM_MODELS):
#         model = GPT2ClassificationHeadModel.from_pretrained(GPT2_MODEL_PATH, clf_dropout=0.2, n_class=len(y_columns))
#         model.load_state_dict(torch.load(paths[i]))
#         model.to(device)
#         for param in model.parameters():
#             param.requires_grad = False
#         model.eval()
    
#         test_preds = np.zeros((len(X_test)))
#         test = torch.utils.data.TensorDataset(torch.tensor(X_test, dtype=torch.long))
#         test_loader = torch.utils.data.DataLoader(test, batch_size=32, shuffle=False)
#         tk0 = tqdm(test_loader)
#         for i, (x_batch,) in enumerate(tk0):
#             pred = model(x_batch.to(device))
#             test_preds[i * 32:(i + 1) * 32] = pred[:, 0].detach().cpu().squeeze().numpy()

#         test_pred = torch.sigmoid(torch.tensor(test_preds)).numpy().ravel()
        
#         gpt2_preds.append(test_pred)
#         gc.collect()
#         torch.cuda.empty_cache()
    
    
#     del tokenizer, test, X_test
#     gc.collect()
#     torch.cuda.empty_cache()
#     return gpt2_preds

In [14]:
# ########################### Bert Large ###########################
# BERT_LARGE_MODEL_DIR_BASE = '../input/tf-large-bert-inference-2/1560187254-20190610t173847z-001'
# BERT_LARGE_MODEL = '/1560187254/'
# start = time.time()
# print("Starting BERT large...")
# bert_large_preds = bert_large(BERT_LARGE_MODEL_DIR_BASE, BERT_LARGE_MODEL, test)
# del bert_large
# gc.collect()
# torch.cuda.empty_cache()
# #following required for bert large - GPU memory doesn't get released.
# from numba import cuda
# cuda.select_device(0)
# cuda.close()
# print("End BERT large, time taken (seconds): ", time.time()-start)
# ########################### Bert Large ###########################

# # ########################### Bert Base ###########################
BERT_BASE_MODEL_PATHS = ['../input/pytorch-bert-base-inference/bert_pytorch_mymodel.bin']  

BERT_BASE_MODEL_PATHS = ['../input/bert-base-5-models/bert_model5folds/bert_model5folds/bert_pytorch_mymodel_fold1.bin',
                        '../input/bert-base-5-models/bert_model5folds/bert_model5folds/bert_pytorch_mymodel_fold2.bin',
                        '../input/bert-base-5-models/bert_model5folds/bert_model5folds/bert_pytorch_mymodel_fold3.bin',
                        '../input/bert-base-5-models/bert_model5folds/bert_model5folds/bert_pytorch_mymodel_fold4.bin',
                        '../input/bert-base-5-models/bert_model5folds/bert_model5folds/bert_pytorch_mymodel_fold5.bin',
                        
                        '../input/2nd-bert-base-5-models/bert_pytorch_mymodel_fold0.bin',
                        '../input/2nd-bert-base-5-models/bert_pytorch_mymodel_fold1.bin',                         
                        '../input/2nd-bert-base-5-models/bert_pytorch_mymodel_fold2.bin',
                        '../input/2nd-bert-base-5-models/bert_pytorch_mymodel_fold3.bin',                         
                        '../input/2nd-bert-base-5-models/bert_pytorch_mymodel_fold4.bin'                         
                        ]


#BERT_BASE_MODEL_PATHS = ['../input/2nd-bert-base-5-models/bert_pytorch_mymodel_fold1.bin']

start = time.time()
print("Starting BERT Base...")
bert_base_preds = bert_base(BERT_BASE_MODEL_PATHS,train,test)
del bert_base
gc.collect()
print("End BERT Base, time taken (seconds): ", time.time()-start)
# ########################### Bert Base ###########################

  

Starting BERT Base...
End BERT Base, time taken (seconds):  3386.538088321686


In [15]:
len(bert_base_preds)

10

In [16]:


########################## GPT2  ###########################
GPT2_MODEL_PATHS = ['../input/gpt2-models-for-ensemble/gpt2_pytorch_mymodel_fold0.bin',
                   '../input/gpt2-models-for-ensemble/gpt2_pytorch_mymodel_fold1.bin',
                   '../input/gpt2-models-for-ensemble/gpt2_pytorch_mymodel_fold2.bin',
                   '../input/gpt2-models-for-ensemble/gpt2_pytorch_mymodel_fold3.bin',
                   '../input/gpt2-models-for-ensemble/gpt2_pytorch_mymodel_fold4.bin']
start = time.time()
print("Starting GPT2...")
gpt2_preds = gpt2_model(GPT2_MODEL_PATHS, train, test)
del gpt2_model
gc.collect()
print("End GPT2, time taken (seconds): ", time.time()-start)
########################## GPT2  ###########################



Starting GPT2...


../input/jigsawdatasets/pytorch-pretrained-gpt2/pytorch-pretrained-gpt2/pytorch_pretrained_bert/tokenization_gpt2.py:146: ResourceWarning: unclosed file <_io.TextIOWrapper name='../input/jigsawdatasets/gpt2-models_kaggle/gpt2-models_kaggle/vocab.json' mode='r' encoding='UTF-8'>
  self.encoder = json.load(open(vocab_file))
../input/jigsawdatasets/pytorch-pretrained-gpt2/pytorch-pretrained-gpt2/pytorch_pretrained_bert/tokenization_gpt2.py:151: ResourceWarning: unclosed file <_io.TextIOWrapper name='../input/jigsawdatasets/gpt2-models_kaggle/gpt2-models_kaggle/merges.txt' mode='r' encoding='utf-8'>
  bpe_data = open(merges_file, encoding='utf-8').read().split('\n')[1:-1]


2560
End GPT2, time taken (seconds):  1771.2205953598022


In [17]:

# ########################### LSTM  ###########################
RNN_PATHS = ['../input/jigsaw-model3/', '../input/rnn-for-ensemble/rnn/']
start = time.time()
print("Starting RNN...")
rnn_preds = rnn_lstm(RNN_PATHS, train, test)
del rnn_lstm
gc.collect()
print("End RNN, time taken (seconds): ", time.time()-start)
########################### LSTM  ###########################
              

Starting RNN...


/opt/conda/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: numpy.ufunc size changed, may indicate binary incompatibility. Expected 192 from C header, got 216 from PyObject
  return f(*args, **kwds)
/opt/conda/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: numpy.ufunc size changed, may indicate binary incompatibility. Expected 192 from C header, got 216 from PyObject
  return f(*args, **kwds)
/opt/conda/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: numpy.ufunc size changed, may indicate binary incompatibility. Expected 192 from C header, got 216 from PyObject
  return f(*args, **kwds)


488836
n unknown words (crawl):  148789
n unknown words (glove):  152310
torch.Size([97320, 300])
End RNN, time taken (seconds):  1295.441802740097


In [18]:
#predictions = (0.35*np.mean(bert_base_preds,axis=0)  + 0.35*bert_large_preds[:,0]  + 0.3*np.mean(rnn_preds,axis=0))

#predictions = (np.mean(bert_base_preds,axis=0)) #+ np.mean(rnn_preds,axis=0))/2 #np.mean(gpt2_preds,axis=0)

# predictions = np.mean(gpt2_preds,axis=0) *23/101
# predictions += np.mean(bert_base_preds,axis=0)*48/101 
# predictions +=  np.mean(rnn_preds,axis=0)  * 30/101

#predictions = (np.mean(bert_base_preds,axis=0))

#28 bert + 27 bert + 16 rnn + 19 gpt2 + 11 rnn) 
predictions = np.mean(bert_base_preds[0:5],axis=0)*28/101.0 + np.mean(bert_base_preds[5:],axis=0)*27/101.0
predictions += rnn_preds[0]*16/101.0 + rnn_preds[1]*11/101.0
predictions += np.mean(gpt2_preds,axis=0)*19/101.0
 

In [19]:
submission = pd.DataFrame.from_dict({
    'id': test['id'],
    'prediction':  predictions
})

submission.to_csv('submission.csv', index=False) 

In [20]:
submission

,id,prediction
0,7000000,0.002139
1,7000001,0.000075
2,7000002,0.002047
3,7000003,0.000771
4,7000004,0.989459
5,7000005,0.000083
6,7000006,0.001023
7,7000007,0.002543
8,7000008,0.016949
9,7000009,0.006536


In [21]:
#pd.DataFrame([np.mean(bert_base_preds,axis=0),np.mean(rnn_preds,axis=0),np.mean(gpt2_preds,axis=0)]).T